<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Explore, segment, and cluster the neighborhoods in the city of Toronto</font></h1>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

# Request and get data from wikipedia

In [52]:
from bs4 import BeautifulSoup
mysource = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(mysource, 'lxml')
table = soup.find_all('table', {'class':'wikitable sortable'})

# Create the data

In [53]:
Toronto_table = pd.read_html(str(table[0]), index_col=None, header=0)[0]
Toronto_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [40]:
type(Toronto_table)

pandas.core.frame.DataFrame

# data cleaning

In [54]:
#  1. Only process the cells that have an assigned borough.
Toronto_table = Toronto_table[Toronto_table.Borough != 'Not assigned']
Toronto_table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [55]:
# 2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Toronto_table.loc[Toronto_table.Neighbourhood == 'Not assigned', 'Neighbourhood'] = Toronto_table.Borough

Toronto_table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [56]:

# 3. Group Neghborhood by Postcode
Toronto_tableN = Toronto_table.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x: ','.join(set(x))).reset_index()

# My data frame

In [57]:
Toronto_tableN.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
Toronto_tableN.shape

(103, 3)

# The latitude and the longitude coordinates of each neighborhood.

In [60]:
Geol = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
Geol.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [61]:
Geol.rename(columns ={'Postal Code':'Postcode'}, inplace = True) 
Geol.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Add the two tables 

In [62]:
Toronto_tableN.reset_index(drop=True, inplace=True)
Geol.reset_index(drop=True, inplace=True)
Toronto_tableN = Toronto_tableN.astype(str)
Geol = Geol.astype(str)
Toronto_tableN = pd.merge(Toronto_tableN, Geol, on = 'Postcode')
Toronto_tableN['Latitude'] = pd.to_numeric(Toronto_tableN['Latitude'], errors='coerce')
Toronto_tableN['Longitude'] = pd.to_numeric(Toronto_tableN['Longitude'], errors='coerce')
Toronto_tableN.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


# Explore and cluster the neighborhoods in Toronto.

In [65]:
c_lat = Toronto_tableN.Latitude.mean()
c_lon = Toronto_tableN.Longitude.mean()
c_lat,c_lon

(43.70460773398059, -79.39715291165048)

In [70]:
from sklearn.cluster import KMeans
import folium # map rendering library

kclusters = 5
toronto_grouped_clustering = Toronto_tableN.drop(['Neighbourhood','Borough','Postcode'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [72]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Toronto_tableN['Cluster Labels'] = kmeans.labels_
map_clusters = folium.Map(location=[c_lat, c_lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_tableN['Latitude'], Toronto_tableN['Longitude'], Toronto_tableN['Neighbourhood'], Toronto_tableN['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster: ' + str(cluster) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [73]:
map_clusters
